In [25]:
import math
import numpy as np
import pandas as pd
import time
import datetime

In [26]:
path = f"data/near_historical.csv"
df_historical= pd.read_csv(path)
# df_historical.dropna(axis=0,inplace=True)
# path = f"drive/Shared drives/Model/Data Extraction/NEAR/ValidationOutput.csv"
# df_outputHistOld = pd.read_csv(path)
# print(df_historical)

In [27]:
df_historical = df_historical[-5:]
df_historical.reset_index(drop=True, inplace=True)

In [28]:
No_Validators = int((len(df_historical.columns)-df_historical.columns.get_loc("block_time_empirical")) / 15)
df_output = pd.DataFrame(columns = ['Epoch Number', 'Validator Name', 'Stake', 'blocks_prod_ratio', 'chunks_prod_ratio','UptimePCT', 'Uptime', 'Actual Rewards', 'Expected Rewards', 'Realized APY', 'Expected APY', 'Absolute Difference in Rewards', 'Absolute Difference in APY', '% Difference in Rewards', 'Rewards Method Used'],
                         index = range(No_Validators))
# This dataframe will be output for the studio file
df_outputHist = pd.DataFrame(columns = ['Epoch Number', 'Validator Name', 'Stake', 'blocks_prod_ratio', 'chunks_prod_ratio','UptimePCT', 'Uptime', 'Actual Rewards', 'Expected Rewards', 'Realized APY', 'Expected APY',
                                        'Absolute Difference in Rewards', 'Absolute Difference in APY', '% Difference in Rewards'])

# print(df_output)
#print(df_outputHist)

In [29]:
for row in range (2, len(df_historical)):
  # Constants from Network
  totalStake                          = float(df_historical['total_staked'][row])*1E-24
  
  totalSupply                         = float(df_historical['total_supply'][row])*1E-24
  Block_time_empirical                = float(df_historical['block_time_empirical'][row]) 
  ONLINE_THRESHOLD_MIN                = float(df_historical['ONLINE_THRESHOLD_MIN'][row])
  ONLINE_THRESHOLD_MAX                = float(df_historical['ONLINE_THRESHOLD_MAX'][row])   
  TREASURY_PCT                        = 0.1
  EPOCHS_PER_YEAR                     = float(df_historical['EPOCHS_A_YEAR'][row])
  EPOCH_LENGTH                        = float(df_historical['EPOCH_LENGTH'][row])
  REWARD_PCT_PER_YEAR                 = 0.05
  BLOCK_PRODUCER_KICKOUT_THRESHOLD    = float(df_historical['BLOCK_PRODUCER_KICKOUT_THRESHOLD'][row])
  CHUNK_PRODUCER_KICKOUT_THRESHOLD    = float(df_historical['CHUNK_PRODUCER_KICKOUT_THRESHOLD'][row])
  E_blockTime                         = float(df_historical['BLOCK_TIME_TARGET'][row])


  # Parameters to be varied by validator
  for i in range(No_Validators):

    df_output['Validator Name'][i]                   = df_historical[f'val_{i}_name'][row]
    df_output['Epoch Number'][i]                     = df_historical['epoch_height'][row]   
    df_output['Stake'][i]                            = float(df_historical[f'val_{i}_sum_stake'][row])*1E-24
    
    num_produced_blocks                = float(df_historical[f'val_{i}_produced_blocks'][row])
    num_expected_blocks                = float(df_historical[f'val_{i}_expected_blocks'][row])
    num_produced_chunks                = float(df_historical[f'val_{i}_produced_chunks'][row])
    num_expected_chunks                = float(df_historical[f'val_{i}_expected_chunks'][row])

    if num_expected_blocks == 0 or num_expected_chunks == 0:
      uptime_Pct_v = 0
      blocks_prod_ratio = 0
      chunks_prod_ratio = 0
    elif num_produced_blocks/num_expected_blocks < BLOCK_PRODUCER_KICKOUT_THRESHOLD/100 or num_produced_chunks/num_expected_chunks < CHUNK_PRODUCER_KICKOUT_THRESHOLD/100:
      uptime_Pct_v = 0
      blocks_prod_ratio = num_produced_blocks/num_expected_blocks
      chunks_prod_ratio = num_produced_chunks/num_expected_chunks
    else:
      uptime_Pct_v = (num_produced_blocks/num_expected_blocks + num_produced_chunks/num_expected_chunks)/2
      blocks_prod_ratio = num_produced_blocks/num_expected_blocks
      chunks_prod_ratio = num_produced_chunks/num_expected_chunks

    df_output['blocks_prod_ratio'][i] = num_produced_blocks/num_expected_blocks
    df_output['chunks_prod_ratio'][i] = num_produced_chunks/num_expected_chunks
    df_output['UptimePCT'][i]         = uptime_Pct_v

    uptime_v                          = min(1,max(0,(uptime_Pct_v-ONLINE_THRESHOLD_MIN) / (ONLINE_THRESHOLD_MAX-ONLINE_THRESHOLD_MIN)))
    df_output['Uptime'][i]            = uptime_v

    E_epochReward           = totalSupply * ( REWARD_PCT_PER_YEAR/EPOCHS_PER_YEAR) *Block_time_empirical/E_blockTime
    E_totalValidatorReward  = (1-TREASURY_PCT)*E_epochReward
    E_validator             = uptime_v * E_totalValidatorReward * df_output['Stake'][i]/totalStake

    E_APY                      = (1+E_validator/df_output['Stake'][i])**(EPOCHS_PER_YEAR*E_blockTime/Block_time_empirical)-1 # POOR MODEL; SHOULD LOOP THROUGH YEAR
        
    staked_before             = float(df_historical[f'val_{i}_sum_stake'][row-1])
    stake_after               = float(df_historical[f'val_{i}_sum_stake'][row])
    unstaked_amount           = float(df_historical[f'val_{i}_unstaked_amount'][row-2])
    added_stake               = float(df_historical[f'val_{i}_added_stake'][row-2])
    actual_rewards            = (stake_after - (staked_before + added_stake - unstaked_amount))/1e24
    Realized_APY              = (1+actual_rewards/df_output['Stake'][i])**(EPOCHS_PER_YEAR*E_blockTime/Block_time_empirical)-1 
    method_used_for_rewards   = 2
    
    df_output['Actual Rewards'][i]   = actual_rewards 
    df_output['Expected Rewards'][i] = E_validator

    if float(df_output['Actual Rewards'][i]) != 0:
      df_output['% Difference in Rewards'][i] = (E_validator - float(df_output['Actual Rewards'][i])) / float(df_output['Actual Rewards'][i]) *100
    else:
      df_output['% Difference in Rewards'][i] = 'NaN'

    df_output['Expected APY'][i]                   = E_APY*100
    df_output['Realized APY'][i]                   = Realized_APY*100    
    df_output['Absolute Difference in Rewards'][i] = E_validator - float(df_output['Actual Rewards'][i])
    df_output['Absolute Difference in APY'][i]     = (df_output['Expected APY'][i]  -   df_output['Realized APY'][i])
    df_output['Rewards Method Used'][i]            = method_used_for_rewards

  #df_output.to_csv(f"drive/Shared drives/Model/Data Extraction/NEAR/ValidationOutput{row}.csv", index = False)
  df_outputHist = df_outputHist.append(df_output, ignore_index = True)

/var/folders/0y/dd7nd74d5ybdsz_l9kpznc300000gn/T/ipykernel_3376/382777319.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_outputHist = df_outputHist.append(df_output, ignore_index = True)


In [30]:
df_outputHist.to_csv(f"val_out.csv", index = False)